In [2]:
import pickle
dataset=open('split_dataset','rb')
dataset=pickle.load(dataset)
testset=open('split_testset','rb')
testset=pickle.load(testset)

In [3]:
def label_get(dataset):
    labelset=[]
    for index,row in dataset.iterrows():
        labelset.append(dataset.loc[index,'label'])
    return labelset

labelset=label_get(dataset)


In [4]:
def pd_li_transform(dataset):
    dataset_list=[]
    for index,row in dataset.iterrows():
        str2=" "
        if dataset.loc[index,'comment_all']is None:
            dataset_list.append(dataset.loc[index,'content'])
        else:
            dataset_list.append(dataset.loc[index,'content']+dataset.loc[index,'comment_all'])
    return dataset_list
testset_list=pd_li_transform(testset)
dataset_list=pd_li_transform(dataset)



In [5]:
def list_deal(dataset,labelset):
    labelset2=[]
    dataset2=[]
    for i in range(len(dataset)):
        if (dataset[i]!=[]):
            dataset2.append(dataset[i])
            labelset2.append(labelset[i])
    return dataset2,labelset2
dataset_list,labelset=list_deal(dataset_list,labelset)

In [6]:
def list_onehot(labelset):
    newli=[]
    for index in labelset:
        if (index==-1):newli.append([-1.0,0.0,0.0])
        if (index==0):newli.append([0.0,-1.0,0.0])
        if (index==1):newli.append([0.0,0.0,-1.0])
    return newli
def list_float(labelset):
    newli=[]
    for index in labelset:
        if (index==-1):newli.append(0.0)
        if (index==0):newli.append(1.0)
        if (index==1):newli.append(2.0)
    return newli
labelset3=list_float(labelset)
labelset2=list_onehot(labelset)

In [7]:
def li_str_transform(dataset_list):
    dataset2=[]
    for seq in dataset_list:
        str1=' '.join(seq)
        dataset2.append(str1)
    return dataset2
dataset_str=li_str_transform(dataset_list)
testset_str=li_str_transform(testset_list)

In [19]:
from fastNLP import DataSet
data_init=DataSet({'raw_chars':dataset_list,'target':labelset})
data_train=data_init[:35000]
data_validation=data_init[35001:]
data_test=DataSet({'raw_chars':testset_list})


In [20]:
from fastNLP import Vocabulary
vocab=Vocabulary()
vocab.from_dataset(data_train,field_name='raw_chars',no_create_entry_dataset=[data_validation])
vocab.index_dataset(data_train,field_name='raw_chars')
vocab.index_dataset(data_validation,field_name='raw_chars')
vocab.index_dataset(data_test,field_name='raw_chars')
target_vocab=Vocabulary(padding=None,unknown=None)
target_vocab.from_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_validation,field_name='target')

Vocabulary([0, -1, 1]...)

In [21]:
data_train


+------------------------------------------+--------+
| raw_chars                                | target |
+------------------------------------------+--------+
| [3721, 3675, 1813, 7477, 3721, 352, 3... | 2      |
| [97, 2, 974, 149, 32, 98, 167, 59, 18... | 1      |
| [15642, 5691, 6, 9217, 7075, 2, 154, ... | 2      |
| [536, 114, 167, 2, 127, 182, 35, 709,... | 1      |
| [10, 21211, 2, 7642, 22, 2, 8062, 226... | 2      |
| [18670, 83, 19401, 492, 378, 141, 506... | 0      |
| [3652, 3620, 2607, 554, 225, 12, 5346... | 0      |
| [145, 220, 3, 5, 47, 134, 2, 15173, 3... | 0      |
| [270, 1200, 77, 39, 1692, 24843, 8876... | 0      |
| [1441, 4317, 129, 4, 7, 175, 5, 274, ... | 2      |
| [7643, 2, 268, 10394, 2, 7171, 15174,... | 2      |
| [88769, 1641, 16692, 2, 121, 45732, 1... | 2      |
| ...                                      | ...    |
+------------------------------------------+--------+

In [22]:
data_train.rename_field('raw_chars','words')
data_validation.rename_field('raw_chars','words')
data_train.set_input('words')
data_train.set_target('target')
data_validation.set_input('words')
data_validation.set_target('target')

+--------+------------------------------------------+
| target | words                                    |
+--------+------------------------------------------+
| 1      | [143, 6, 1071, 73, 240, 700, 328, 441... |
| 0      | [11692, 251, 39496, 2749, 59995, 34, ... |
| 0      | [158503, 1188, 16448, 25969, 420, 2, ... |
| 2      | [4159, 236, 518, 2, 153, 81620, 2, 69... |
| 0      | [1093, 1586, 230, 199, 2424, 3228, 27... |
| 0      | [683, 2707, 46321, 9399, 3, 51, 3, 54... |
| 1      | [616, 64, 40, 229, 3, 794, 67, 1265, ... |
| 0      | [328, 104, 1339, 3783, 1148, 2496, 31... |
| 2      | [9841, 10021, 15083, 14882, 19515, 98... |
| 0      | [5048, 368, 126, 19264, 626, 109, 209... |
| 0      | [574, 3553, 120, 280, 128, 960, 1225,... |
| 2      | [7836, 32, 5018, 921, 167, 1516, 135,... |
| ...    | ...                                      |
+--------+------------------------------------------+

In [23]:
print(data_validation)

+--------+------------------------------------------+
| target | words                                    |
+--------+------------------------------------------+
| 1      | [143, 6, 1071, 73, 240, 700, 328, 441... |
| 0      | [11692, 251, 39496, 2749, 59995, 34, ... |
| 0      | [158503, 1188, 16448, 25969, 420, 2, ... |
| 2      | [4159, 236, 518, 2, 153, 81620, 2, 69... |
| 0      | [1093, 1586, 230, 199, 2424, 3228, 27... |
| 0      | [683, 2707, 46321, 9399, 3, 51, 3, 54... |
| 1      | [616, 64, 40, 229, 3, 794, 67, 1265, ... |
| 0      | [328, 104, 1339, 3783, 1148, 2496, 31... |
| 2      | [9841, 10021, 15083, 14882, 19515, 98... |
| 0      | [5048, 368, 126, 19264, 626, 109, 209... |
| 0      | [574, 3553, 120, 280, 128, 960, 1225,... |
| 2      | [7836, 32, 5018, 921, 167, 1516, 135,... |
| ...    | ...                                      |
+--------+------------------------------------------+


In [24]:
import torch
from fastNLP.embeddings import BertEmbedding
from fastNLP import Vocabulary
from fastNLP.core.losses import MSELoss
embed=BertEmbedding(vocab,model_dir_or_name='cn-wwm', include_cls_sep=True)

loading vocabulary file C:\Users\LENOVO\.fastNLP\embedding\bert-chinese-wwm\vocab.txt
Load pre-trained BERT parameters from file C:\Users\LENOVO\.fastNLP\embedding\bert-chinese-wwm\chinese_wwm_pytorch.bin.
Bert Model will return 1 layers (layer-0 is embedding result): [-1]


In [25]:
from fastNLP.models import BertForSequenceClassification
model = BertForSequenceClassification(embed,num_labels=3)


In [26]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric, Adam


device = 0 if torch.cuda.is_available() else 'cpu'
trainer = Trainer(data_train, model,
                  optimizer=Adam(model_params=model.parameters(), lr=2e-5),
                  loss=CrossEntropyLoss(), device=device,
                  batch_size=8, dev_data=data_validation,
                  metrics=AccuracyMetric(), n_epochs=2, print_every=1)
trainer.train()

input fields after batch(if batch size is 2):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 247]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2021-11-26-01-36-39-787608


RuntimeError: After split words into word pieces, the lengths of word pieces are longer than the maximum allowed sequence length:512 of bert. You can set `auto_truncate=True` for BertEmbedding to automatically truncate overlong input.

In [ ]:

DataSetIter(dataset, batch_size=batch_size, sampler=None)

In [ ]:
word1=torch.LongTensor(vocab.to_index("男"))
word2=torch.LongTensor(vocab.to_index("女"))
word3=torch.LongTensor(vocab.to_index("热血"))
#word1=embed(word1)
#word2=embed(word2)
#word3=embed(word3)
print(word1.size())
print(word2.size())
print(word3.size())


torch.Size([475])
torch.Size([333])
torch.Size([3675])
